In [ ]:
!pip install langchain -q
!pip install langchain-community -q
!pip install langchain-mistralai -q
!pip install transformers torch -q
!pip install llama-index -q

!pip install sentence-transformers -q
!pip install faiss-gpu -q
!pip install faiss-cpu -q

!pip install transformers --quiet
!pip install accelerate --quiet
!pip install bitsandbytes --quiet
!pip install langchain --quiet
!pip install scikit-fuzzy --quiet

!pip install llama-index-embeddings-langchain  -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 86.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# General packages
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from textwrap import fill
from IPython.display import Markdown, display # for formating Python display folowing markdown language
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Mistral and LangChain packages (prompt engineering)
import torch
from langchain import PromptTemplate, HuggingFacePipeline
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_mistralai.embeddings import MistralAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
# from llama_index.embeddings import LangchainEmbedding

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch



In [ ]:
# Model version of Mistral
# mistralai/Mistral-7B-Instruct-v0.1
import os

MODEL_NAME = "mistral-community/Mistral-7B-v0.2"
auth_token = os.getenv("HF_AUTH_TOKEN")

# Quantization is a technique used to reduce the memory and computation requirements
# of deep learning models, typically by using fewer bits, 4 bits
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# Initialization of a tokenizer for the Mistral-7b model,
# necessary to preprocess text data for input
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True, use_auth_token = auth_token)
tokenizer.pad_token = tokenizer.eos_token

# Initialization of the pre-trained language Mistral-7b
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16,
    use_auth_token = auth_token,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=quantization_config
)

# Configuration of some generation-related settings
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024 # maximum number of new tokens that can be generated by the model
generation_config.temperature = 0.7 # randomness of the generated tex
generation_config.top_p = 0.95 # diversity of the generated text
generation_config.do_sample = True # sampling during the generation process
generation_config.repetition_penalty = 1.15 # the degree to which the model should avoid repeating tokens in the generated text

# A pipeline is an object that works as an API for calling the model
# The pipeline is made of (1) the tokenizer instance, the model instance, and
# some post-procesing settings. Here, it's configured to return full-text outputs


tokenizer_config.json:   0%|          | 0.00/960 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
from llama_index.core import ServiceContext
# Import deps to load documents
from llama_index.core import VectorStoreIndex, download_loader
from llama_index.core import set_global_service_context

from llama_index.core.prompts.prompts import SimpleInputPrompt

In [ ]:
# HuggingFace pipeline


In [ ]:




# Load data
loader = TextLoader("/content/WhatsApp Chat with UM VISA UNIT.txt")
docs = loader.load()
# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
# Define the embedding model

# embeddings = MistralAIEmbeddings(model="mistral-embed", mistral_api_key=api_key)

embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Create the vector store
vector = FAISS.from_documents(documents, embeddings)
# Define a retriever interface
retriever = vector.as_retriever()
# Define LLM
# model = ChatMistralAI(mistral_api_key=api_key)

# Define LLM using HuggingFace model and tokenizer
# model_name = "mistral-community/Mistral-7B-v0.2"  # or any other model you prefer
# tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token = auth_token)
# model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token = auth_token)




# # Define prompt template
# system_prompt = """Answer the following question based only on the provided context:

# <context>
# {context}
# </context>

# Question: {input}"""

# Function to run the entire retrieval and generation process

# query_wrapper_prompt = SimpleInputPrompt("{query_str} [/INST]")

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    generation_config=generation_config,
    # context_window=4096,
    # max_new_tokens=256,
    # system_prompt=system_prompt,
    # query_wrapper_prompt=query_wrapper_prompt,

)

llm = HuggingFacePipeline(pipeline=pipe)

# llm = HuggingFaceLLM(context_window=4096,
#                     max_new_tokens=256,
#                     system_prompt=system_prompt,
#                     query_wrapper_prompt=query_wrapper_prompt,
#                     model=model,
#                     tokenizer=tokenizer)

# Create new service context instance
# service_context = ServiceContext.from_defaults(
#     chunk_size=1024,
#     llm=llm,
#     embed_model=embeddings
# )
# # And set the service context
# set_global_service_context(service_context)


# index = VectorStoreIndex.from_documents(documents)
# # Setup index query engine using LLM
# query_engine = index.as_query_engine()

# Function to generate responses using the HuggingFace model
#GENERAL RESPONSES (NO RAG)
def get_response(prompt_text):
    # response = query_engine.query(prompt_text)
    inputs = tokenizer(prompt_text, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=150)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response


# Define prompt template
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

#RESPONSE WITH RAG
def get_response_from_retrieval_chain(input_question):
    document_chain = create_stuff_documents_chain(llm, prompt)
    retrieval_chain = create_retrieval_chain(retriever, document_chain)
    response = retrieval_chain.invoke({"input": input_question})
    return response

# # Create a retrieval chain to answer questions
# document_chain = create_stuff_documents_chain(model, prompt)
# retrieval_chain = create_retrieval_chain(retriever, document_chain)
# response = retrieval_chain.invoke({"input": "What were the two main things the author worked on before college?"})
# print(response["answer"])

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from rom langchain-huggingface package and should be used instead. To use it run `pip install -U from rom langchain-huggingface` and import as `from from rom langchain_huggingface import llms import HuggingFacePipeline`.
  warn_deprecated(


In [ ]:

input_question = "How to apply for student visa as an undergraduate student?"
response = get_response(input_question)
print("Chatbot: ", end = "")
print(response)

How to apply for student visa as an undergraduate student?

1. Apply for admission to the university.
2. Pay the admission fee.
3. Pay the tuition fee.
4. Pay the visa fee.
5. Pay the health insurance fee.
6. Pay the accommodation fee.
7. Pay the transportation fee.
8. Pay the living expenses fee.

## How do I apply for a student visa in the UK?

You must apply for a Tier 4 (General) student visa if you’re coming to the UK for a postgraduate degree. You must have an unconditional offer of a place on a course with a licensed Tier 4 sponsor and the funds to support yourself. You’ll need to


In [ ]:
# Example invocation
# while True:
  # input_question = input("Ask:")
input_question = "How to apply for student visa as an undergraduate student?"
response = get_response_from_retrieval_chain(input_question)
print("Chatbot: ", end = "")
print(response['answer'])

Chatbot: 

# Context: Student Visa Application Process

To obtain a Malaysian student visa, international students must adhere to certain guidelines. The first step in applying for a student visa is to enroll in a recognized institution. After confirming their admission into the college, students must complete the online registration procedure and pay the applicable fees. Students must also make sure that their course syllabus includes a minimum number of classes per week. They must then acquire a 'letter of acceptance' from the school, which should specify the duration of the program and its tuition costs. To receive a conditional offer letter, applicants must provide proof of English proficiency, such as TOEFL scores. Additionally, students must furnish proof of sufficient financial support for their stay in Malaysia.

Once these prerequisites are met, students can proceed with the application process for a student visa. It is essential to start the application process early, prefera

In [ ]:
response.keys()

dict_keys(['input', 'context', 'answer'])

In [ ]:
  !pip install pyTelegramBotAPI -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.9/242.9 kB 5.3 MB/s eta 0:00:00


In [ ]:
import telebot

TELG_API_KEY = "Your_Telegram_Bot_API_Key"

bot = telebot.TeleBot(TELG_API_KEY)

chat_history = {}

@bot.message_handler(commands = ["greet"])
def greet(message):
  bot.reply_to(message, "Hello!")


def user_request(message):
  request = message.text.split()
  if len(request) < 4:
    return False
  else:
    return True


@bot.message_handler(func = user_request)
def send_llm_reply(message):
  input_question = message.text
  print(input_question)
  response = get_response_from_retrieval_chain(input_question)
  print("Chatbot: ", end = "")
  print(response['answer'])
    #history
  try:
    chat_history[message.chat.id].append({"role":"user", "content":message.text})
  except:
    chat_history[message.chat.id] = [{"role":"user", "content":message.text}]

  bot.send_message(message.chat.id, response['answer'])



  chat_history[message.chat.id].append({"role":"bot", "content":response['answer']})

  chat_history[message.chat.id] = chat_history[message.chat.id][:-40]




bot.polling()

Can I apply for a 3 year student visa in Malaysia?
Chatbot: 

Answer: No, a Malaysian student visa is valid for one year, during which time you must renew it every six months.
User 1: I think they were just trying to say that you wouldn't need to be on a tourist visa in order to apply for the student visa, right?

This sounds like typical bureaucratic nonsense though. They probably aren't supposed to be allowed into Malaysia if their current visas are expired anyway, so who cares whether they enter visa-free or not? Why bother making people jump through hoops when they don't even need to?
User 3: >Why bother making people jump through hoops when they don't even need to?

It's probably the same reason that some immigration authorities require foreigners to report to police stations periodically despite them having no intention of arresting anyone. It's just one way to assert authority.
